In [9]:
import math
import gurobipy as gp
from gurobipy import GRB, quicksum

# Data
farms = [0,1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
East = [0,-3, 1, 4, -5, -5, -4, 6, 3, -1, 0, 6, 2, -2, 6, 1, -3, -6, 2, -6, 5]
North = [0,3, 11, 7, 9, -2, -7, 0, -6, -3, -6, 4, 5, 8, 10, 8, 1, 5, 9, -5, -4]
frequency = [None,"Every day", "Every day", "Every day", "Every day", "Every day", "Every day", "Every day", "Every day", "Every day", "Every other day",
             "Every other day", "Every other day", "Every other day", "Every other day",
             "Every other day", "Every other day", "Every other day", "Every other day", "Every other day", "Every other day"]
collection = [None,5000, 4000, 3000, 6000, 7000, 3000, 4000, 6000, 5000, 4000, 7000, 3000, 4000, 5000, 6000, 8000, 5000, 7000, 6000, 6000]

# defining Parameters
position = {farm: (East[i], North[i]) for i, farm in enumerate(farms)} #looping each position east to north
milk_required = {farm: collection[i] for i, farm in enumerate(farms) if collection[i] is not None}#looping for every farm
every_day = [farm for i, farm in enumerate(farms) if frequency[i] == "Every day"] #separating every day from every other day and looping thru each category
every_other_day = [farm for i, farm in enumerate(farms) if frequency[i] == "Every other day"]
tanker_capacity = 80000  # Tanker capacity in gallons

# Calculating Euclidean Distance 
def distance(location1, location2):
    return math.sqrt((location1[0] - location2[0])**2 + (location1[1] - location2[1])**2)

distances = {(i, j): distance(position[i], position[j]) for i in farms for j in farms if i != j}

# creating the model 
m = gp.Model("Milk_Collection")

# creating Decision variables
x = m.addVars(distances.keys(), vtype=GRB.BINARY, name="x")  # Binary variables for routes
y = m.addVars(farms, vtype=GRB.BINARY, name="y")            # Binary variables for farm visits
z = m.addVars(farms, vtype=GRB.CONTINUOUS, name="z")        # Continuous variables for milk collected

# The Objective function is to Minimize total distance
m.setObjective(quicksum(distances[i, j] * x[i, j] for i, j in distances.keys()), GRB.MINIMIZE)

# creating Constraints
# Flow conservation for farms constraint
for i in farms:
    if i != 0:  # Not the processing facility
        m.addConstr(quicksum(x[i, j] for j in farms if j != i) == y[i])
        m.addConstr(quicksum(x[j, i] for j in farms if j != i) == y[i])

# Processing facility constraints
m.addConstr(quicksum(x[0, j] for j in farms if j != 0) == 1)  # Start from processing facility
m.addConstr(quicksum(x[j, 0] for j in farms if j != 0) == 1)  # End at processing facility

# collection of milk constraints
m.addConstr(quicksum(z[i] for i in farms) <= tanker_capacity)  # Total milk must not exceed capacity
for i in farms:
    if i in milk_required:
        m.addConstr(z[i] <= milk_required[i] * y[i])  # Milk collected at farm i if visited

# Subtour elimination constraints
s = m.addVars(farms, vtype=GRB.CONTINUOUS, name="s")  # Subtour variables
for i, j in distances.keys():
    if i != 0 and j != 0 and i != j:
        m.addConstr(s[i] - s[j] + truck_capacity * x[i, j] <= truck_capacity - milk_required.get(j, 0))

# Loop for  all "every day" farms
for i in every_day:
    m.addConstr(y[i] == 1)

# Optimize
m.optimize()

# Print results
if m.status == GRB.OPTIMAL:
    print("Optimal routes:")
    total_distance = 0  # To keep track of the total distance

    for i, j in distances.keys():
        if x[i, j].x > 0.5:  # Check if the route is used in the solution
            route_distance = distances[i, j]  # Distance for this route
            total_distance += route_distance  # Add to the total distance
            print(f"Route from {i} to {j}: Distance = {route_distance:.2f}")

    print(f"\nFinal Total Distance Traveled: {total_distance:.2f}")
else:
    print("No optimal solution found.")




Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22621.2))

CPU model: Intel(R) Core(TM) i7-8650U CPU @ 1.90GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 452 rows, 483 columns and 2090 nonzeros
Model fingerprint: 0xf7e5c322
Variable types: 42 continuous, 441 integer (441 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+04]
  Objective range  [1e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+04]
Presolve removed 30 rows and 32 columns
Presolve time: 0.01s
Presolved: 422 rows, 451 columns, 2002 nonzeros
Variable types: 20 continuous, 431 integer (431 binary)

Root relaxation: objective 5.052489e+01, 53 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   50.52489    0   38         